# Facebook data 📈

Directory selection

In [ ]:
from PyQt5 import QtWidgets

import json
import os
from pathlib import Path 

def get_directory():
    app = QtWidgets.QApplication([dir])
    return QtWidgets.QFileDialog.getExistingDirectory(None, "Select messages directory")

dirname = get_directory()

conversation_dirs = os.listdir(dirname)
print('Printing conversations...')
print(conversation_dirs)

First of all we need to fix encoding of message.json files. To do this, we will encode message to **latin-1** and decode to **utf8**. Fixed Json files are created in tmp folder in root folder of jupyter notebook file.

In [ ]:
if not os.path.exists('tmp'):
    os.makedirs('tmp')
    
for convdir in conversation_dirs:
        src = '{}/{}/message.json'.format(dirname, convdir)
        dst = 'tmp/{}.json'.format(convdir)
        print(src)
        
        try:
            src_file = open(src, 'r')
        except Exception:
            continue
        
        jsonToFix = json.load(src_file)
            
        for i in jsonToFix['messages']:
            try:
                i['sender_name'] = i['sender_name'].encode('latin-1').decode('utf8')
                i['content'] = i['content'].encode('latin-1').decode('utf8')
            except Exception:
                pass
        
        jsonToFix['title'] = jsonToFix['title'].encode('latin-1').decode('utf8')

        with open(dst, 'w', encoding='utf8') as fout:
            json.dump(jsonToFix, fout, indent=2, ensure_ascii=False)

        print('File created at {}'.format(dst))

Count messages in all conversations

In [ ]:
total = 0
conversation_counts = {}

filtered_total = 0
filtered_counts = {}

fixed_jsons = os.listdir('tmp')

for jsfile in fixed_jsons:
    fixed_json = json.load(open('tmp/{}'.format(jsfile), 'r', encoding='utf-8'))
    
    count = len(fixed_json["messages"])
    total += count
    
    conversation_counts[str(fixed_json["title"])] = count
    if count < 100:
        filtered_total += count
    else:
        filtered_counts[str(fixed_json["title"])] = count

filtered_counts["Other"] = filtered_total
    
print('Count of messages across conversations:{}'.format(total))
print(conversation_counts)
print(filtered_counts)

Export data to csv files for now

In [ ]:
with open('raw.csv','w',encoding='utf-8') as raw_out:
    for k,v in conversation_counts.items():
        raw_out.write('{};{}\n'.format(k,v))
    
with open('filtered.csv','w',encoding='utf-8') as fil_out:
    for k,v in filtered_counts.items():
        fil_out.write('{};{}\n'.format(k,v))
        
print("Done writing files")